# Introduction to Python in GIS
by: [Bo Fethe](https://bofethe.github.io)

https://github.com/bofethe/python-training

Pre-reqs:
+ ArcGIS Pro

## Setup

Out of the box, ESRI gives you a [conda](https://anaconda.org/Esri) python environment that functions similar to Anaconda's base environment. This means that you should not (and in this case, cannot) edit this environment or you risk running into issues later such as with cloned environment dependencies.

The first thing you should do is clone this environment. Open ArcGIS Pro and navigate to `Project` > `Package Manager` and in the `Active Environment` box click on the `gear icon` > `...` next to the default python environment > `Clone`, and then give your new environment a name and path. After it's cloned, activate your new environment by right clicking it and selecting `Activate`. Now that you have this setup, you can install other libraries, update libraries, break it, fix it, ...whatever.  You can easily default back to the base if something goes wrong.

The more common approach to creating python environments using conda is through a terminal. To clone an environment, open a terminal after initializing your conda and run `conda create -n gis --clone arcgispro-py3` to clone the default esri python environment with a new name of *gis*, followed by `conda activate gis` to activate it.  And if you need to install and additional libraries, run `conda install geopandas` to install the library in the currently activate environment.

ESRI has 2 important geospatial python libraries to be familiar with - `arcpy` for accessing local files and geoprocessing tools, and `arcgis` for accessing web layers and cloud-based geoprocessing tools.

In [1]:
# ESRI libraries
import arcgis
import arcpy

# Non-ESRI libraries
import os
import sys

Print the current python version and authenticate yourself through the active portal in ArcGIS Pro. Alternatively, you can pass your `url`, `username` and `password` as variables.

In [2]:
print('Python version:', sys.version)

gis = arcgis.GIS('Pro')
print('Logged in as:', gis.properties.user.username)

Python version: 3.11.8 (main, Mar 22 2024, 13:25:41) [MSC v.1938 64 bit (AMD64)]
Logged in as: bo.fethe.hc


Create a workspace

In [3]:
arcpy.env.overwriteOutput = True    # Allow overwriting layers
gdbName = 'python-gis' + '.gdb'     # Geodatabase name must include file extension

# Create a new geodatabase.
arcpy.management.CreateFileGDB(out_folder_path='data',
                               out_name=gdbName)

# Set workspace to the new geodatabase
arcpy.env.workspace = os.path.join('data', gdbName)

## Accessing Web Layers

You can access web layers through [arcgis](https://developers.arcgis.com/python/api-reference/). You can either search by name, use the service URL, or by ArcGIS Online item ID as shown below. This layer can be added to an interactive map object and queried as a spatial dataframe. This example is accessing a web layer of Florida's counties with FDOT District ID.

In [13]:
# Define the service URL
districtsID = '0e28ef312008491aa86f90bd9ca7c706'

# Construct the FeatureLayer object 
districts_flc = gis.content.get(districtsID)
districts_layer = districts_flc.layers[0]

## Create a map object and add the layer to it
mapp_esri = gis.map('Florida')
mapp_esri.add_layer(districts_layer)
mapp_esri

MapView(layout=Layout(height='400px', width='100%'))

Read the district layer as a spatial dataframe.  There are many similarities between an arcgis spatial dataframe and a pandas dataframe. For example: you can filter the dataframe by passing in a boolean arguement and the indexes where `TRUE` will be returned. A noteable difference is the geometry type field which stores the vertices in the Well Known Text (WKT) format. The below example filters the districts county layer to only those in FDOT District 7.

In [5]:
# Construct the dataframe and show data type
districtsSDF = districts_layer.query().sdf
print('districtsSDF data type:', type(districtsSDF))

# Filter and show dataframe as a view
districtsSDF[districtsSDF['FDOTDistr'] == 7]

districtsSDF data type: <class 'pandas.core.frame.DataFrame'>


,OBJECTID,NAME,FIRST_FIPS,OBJECTID_1,FEATURE_AR,FEATURE_LE,FDOTDistr,DistrictString,DEPCODE,DEPCODENUM,DCode,County,FDOTCountyCode,GlobalID,Shape__Area,Shape__Length,SHAPE
18,19,Pasco,101,51,0.0,0.0,7,Seven,51,51,D7,Pasco,14,9ae8e5c7-90f4-413c-8f9d-9719e9231a30,2565226674.0,334805.697686,"{""rings"": [[[-9207770, 3292091], [-9207695, 32..."
19,20,Pinellas,103,52,0.0,0.0,7,Seven,52,52,D7,Pinellas,15,2252cc96-be3f-440d-add2-d4668af0333c,963014893.5,944662.124143,"{""rings"": [[[-9220234, 3237823], [-9220096, 32..."
37,38,Citrus,017,9,0.0,0.0,7,Seven,9,9,D7,Citrus,02,05ba6348-13f0-4b94-9f84-051bafb21559,2122850251.5,1241787.556099,"{""rings"": [[[-9181660, 3382364], [-9181551, 33..."
54,55,Hernando,053,26,0.0,0.0,7,Seven,27,27,D7,Hernando,08,d3c764ad-bea2-4c80-bc03-661ed9c0896e,1652400714.5,390262.068123,"{""rings"": [[[-9203518, 3303737], [-9203657, 33..."
56,57,Hillsborough,057,28,0.0,0.0,7,Seven,29,29,D7,Hillsborough,10,38a9e448-4d03-49d0-866b-e6844580a045,3566412953.5,688102.458401,"{""rings"": [[[-9212552, 3195753], [-9212586, 31..."


In [6]:
# Show the data types
districtsSDF.dtypes

OBJECTID                   Int64
NAME              string[python]
FIRST_FIPS        string[python]
OBJECTID_1                 Int32
FEATURE_AR               Float64
FEATURE_LE               Float64
FDOTDistr                  Int32
DistrictString    string[python]
DEPCODE           string[python]
DEPCODENUM                 Int32
DCode             string[python]
County            string[python]
FDOTCountyCode    string[python]
GlobalID          string[python]
Shape__Area              Float64
Shape__Length            Float64
SHAPE                   geometry
dtype: object

## Local Geoprocessing Tools

You can access the geoprocesing tools you would normally have access to in ArcGIS Pro using the [arcpy](https://pro.arcgis.com/en/pro-app/latest/arcpy/functions/alphabetical-list-of-arcpy-functions.htm) library. This example below is exporting the FDOT Districts layer into our default workspace and then confirming the layers in the geodatabase.

In [7]:
# Export the web layer
arcpy.conversion.ExportFeatures(in_features=districtsSDF,
                                out_features='FDOT_Districts_poly')
# Make sure the layer exported
print('Local files:', arcpy.ListFeatureClasses(), sep='\n')

Local files:
['FDOT_Districts_poly']


You can also run geoprocessing tools directly from local files. Similar to in ArcGIS Pro, `arcpy` defaults to the current workspace and can take in relative paths.  You can also pass in absolute paths, but note that this method can cause errors when sharing scripts if the reciever doesn't have access to the absolute path you provided. The below example is converting our FDOT districts polygon layer to a point layer and saving it into the default workspace using relative paths.

In [8]:
# Convert polygon to point
arcpy.management.FeatureToPoint(in_features = 'FDOT_Districts_poly',
                                out_feature_class = 'FDOT_Districts_point',
                                point_location = 'CENTROID')

# Make sure the layer exported
print('Local files:', arcpy.ListFeatureClasses(), sep='\n')

Local files:
['FDOT_Districts_poly', 'FDOT_Districts_point']


## Open Source Options

If you don't have an ESRI license, there are open-source solutions you can use. `GeoPandas` can be used for dataframe operations and `folium` to create maps using leaflet. Note that these do not come in the ESRI python environment by default and will need to be installed.

In [9]:
# Open source libraries
import geopandas as gpd
import folium

Make a geodataframe and put it on a map.

In [10]:
geojson_url = 'https://services.arcgis.com/apTfC6SUmnNfnxuF/arcgis/rest/services/Commissioner_Districts/FeatureServer/0/query?where=1=1&f=pgeojson'

hc_districts = gpd.read_file(geojson_url)
hc_districts

,DISTRICT,geometry
0,1,"POLYGON ((-82.58256 28.05639, -82.58256 28.055..."
1,2,"POLYGON ((-82.62865 28.17327, -82.62482 28.173..."
2,3,"POLYGON ((-82.43602 28.08041, -82.43471 28.080..."
3,4,"POLYGON ((-82.12481 28.17215, -82.10586 28.171..."


In [11]:
#create the map from a layer
mapp_gpd = hc_districts.explore(name = 'Hillsborough Commissioner Districts', column='DISTRICT')

#add a layer selector
folium.LayerControl().add_to(mapp_gpd)

#show the map
mapp_gpd

Create a leaflet map using folium

In [12]:
#create the map defaulting to Hillsborough County
mapp_fol = folium.Map(location=[27.8, -82.3], zoom_start=10)

#add a layer in geojson format
folium.GeoJson(geojson_url, name="Hillsborough Commissioner Districts", 
                popup=folium.GeoJsonPopup(
                    fields=["DISTRICT"],
                    aliases=["Commissioner District:"]
        )).add_to(mapp_fol)

#show the map
mapp_fol

If you enjoyed this tutorial, please let me know! Also, if you have any suggestions on improvements or new topics, open an Issue in GitHub.